In [1]:
!pip install pandas numpy 


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd

# Read roads csv file

In [8]:
# detailed_trajectory = pd.read_pickle("./data_with_trajectory_20s/courier_detailed_trajectory_20s.pkl.xz",compression="xz")
# road_df=pd.read_csv("../LaDe/road-network/roads.csv",sep="\t")

road_df=pd.read_csv("../processed/roads_translated_motherfile.csv")
road_df.head(3)

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,city,geometry,city_chinese
0,4296533,5141,service,NaN,NaN,B,0,0,F,F,Hangzhou,LINESTRING (13373499.140421594 3538916.2947984...,杭州市
1,4296592,5115,tertiary,朝晖路,NaN,B,0,0,F,F,Hangzhou,LINESTRING (13376291.100042382 3539415.0794264...,杭州市
2,4298707,5114,secondary,体育场路,NaN,F,0,0,F,F,Hangzhou,LINESTRING (13375063.92510783 3538697.18123532...,杭州市


# read delivery_city dataset

In [10]:
delivery = pd.read_csv("../LaDe/delivery/delivery_hz.csv")
delivery.head(3)

,order_id,region_id,city,courier_id,lng,lat,aoi_id,aoi_type,accept_time,accept_gps_time,accept_gps_lng,accept_gps_lat,delivery_time,delivery_gps_time,delivery_gps_lng,delivery_gps_lat,ds
0,583722,0,Hangzhou,175,120.17895,30.26401,749,1,10-30 09:20:00,10-30 09:20:00,120.20600,30.28657,10-30 10:30:00,10-30 10:30:00,120.17908,30.26392,1030
1,2819059,0,Hangzhou,175,120.17899,30.26336,749,1,10-31 09:47:00,10-31 09:47:00,120.20591,30.28651,10-31 10:40:00,10-31 10:40:00,120.17884,30.26360,1031
2,2879432,0,Hangzhou,175,120.17896,30.26404,749,1,10-22 10:11:00,10-22 10:11:00,120.20598,30.28668,10-22 15:03:00,10-22 15:03:00,120.17939,30.26395,1022


In [13]:
trajectory20s = pd.read_pickle("../LaDe/data_with_trajectory_20s/courier_detailed_trajectory_20s.pkl.xz",compression="xz")

trajectory20s.head(3)
# traj_df["gps_time"]=pd.to_datetime(traj_df["gps_time"])

# traj_gsort_df = (
#     traj_df.groupby("postman_id",group_keys=False)
#     .apply(lambda g:g.sort_values("gps_time"))
#     .reset_index(drop=True)
# )



# traj_basegroup_df = (
#     traj_df.groupby(["ds","postman_id"])
#     .size()
#     .reset_index(name="count_coordinates")
# )



,ds,postman_id,gps_time,lat,lng
0,321,106f5ac22cfd1574b196d16fed62f90d,03-21 07:31:58,3.953700e+06,3.053400e+06
1,321,106f5ac22cfd1574b196d16fed62f90d,03-21 07:32:18,3.953700e+06,3.053398e+06
2,321,106f5ac22cfd1574b196d16fed62f90d,03-21 07:32:41,3.953700e+06,3.053398e+06


In [9]:
road_df.columns

# osm_id = openstreetmap_id 
# 'code' = osm internal code schema 
# 'fclass', =  functional category of road -  motorway, primary, secondary, residential
#  'name', = official name availble in osm
#  'ref'   = Reference code (like highway numbers, e.g., “G60” for a national expressway). ex. S102 
#  'oneway', = Indicates if the road is one-way (yes/no). Critical for routing algorithms.
#  'maxspeed'= Maximum legal speed limit on the road segment (km/h).
#  'layer' = Vertical layer of the road (used when roads overlap, e.g., flyovers vs. tunnels) --> 
#  'bridge'= Boolean flag (yes/no) indicating if the segment is a bridge.
#  'tunnel'= Boolean flag (yes/no) indicating if the segment is a tunnel.
#  'city', = The city to which the road belongs (Shanghai, Hangzhou, Chongqing, Jilin, Yantai).
# 'geometry' = The polyline coordinates (latitude/longitude pairs) describing the exact shape of the road segment.

Index(['osm_id', 'code', 'fclass', 'name', 'ref', 'oneway', 'maxspeed',
       'layer', 'bridge', 'tunnel', 'city', 'geometry', 'city_chinese'],
      dtype='object')

In [ ]:
# traj_gsort_df["seq"]=(
#     traj_gsort_df.groupby("postman_id")[["lat","lng"]]
#     .apply(lambda g: (g != g.shift())).any(axis=1).cumsum()
#     .reset_index(level=0, drop=True)
# )


In [ ]:
# traj_spacegroup_df = (
#     traj_gsort_df.groupby(["postman_id","seq"])
#     .agg(
#         lat = ("lat","first"),
#         lng = ("lng","first"),
#         start_time = ("gps_time","min"),
#         end_time = ("gps_time","max"),
#         n_pings = ("gps_time","size")
#     )
#     .reset_index()
# )

In [ ]:
# traj_spacegroup_df.to_pickle("trajectory_spacegroup.pkl")